# SCRAPING DE PAGE DYNAMIQUE : SELENIUM


*OBJECTIF*: 
  - Utiliser le module **Selenium** de Python pour scraper les pages dynamiques

Prérequis: 
  - Installer `selenium`
  - Télécharger `driver` selon le navigateur de votre choix (l’objet qui gère le navigateur utilisé par Selenium)  

## Introduction

**BeautifulSoup** est une excellente bibliothèque pour récupérer des données sur le Web, mais elle ne traite pas du contenu créé de manière dynamique. Ce n’est en aucun cas une critique - Beautiful Soup fait exactement le travail qu’elle est censée faire et cela n’inclut pas le rendu de la page Web comme le ferait un navigateur.

Afin d'obtenir ce contenu dynamique, la page Web doit être interprétée par un navigateur afin que le Javascript qui crée le contenu dynamique puisse faire son travail. Mais comment arriver au code HTML rendu par le navigateur? Une réponse consiste à utiliser un navigateur auxiliaire et la bibliothèque **Selenium** de Python. 

Nous allons utiliser un fichier HTML très simple qui contient du texte rendu dynamiquement. 

Copiez le code suivant dans un fichier nommé *test.html*

<html>
<head>
   <title>Page Dynamique</title>
</head>
<body>
   <div id="text">Zone de texte</div>
   <script>
      document.getElementById("text").innerHTML="Ceci est notre texte"
   </script>
</body>
</html>

Tout ce que nous avons ici est un fichier HTML avec un seul `<div>` dans le corps qui contient du texte («Zone de texte») mais lorsque la page est chargée, ce texte est remplacé par le texte généré par le Javascript plus bas. Le script localise le `<div>` par son id, `text`, puis définit le texte interne sur: «Ceci est notre texte».

Maintenant voyons ce qui se passe lorsque nous essayons de scraper cette page

In [1]:
from bs4 import BeautifulSoup

# Lecture du fichier
soup_file=open("test.html")
# Parcourrir le code source
soup = BeautifulSoup(soup_file)

print(soup.prettify)


<bound method Tag.prettify of <html>
<head>
<title>Page Dynamique</title>
</head>
<body>
<div id="text">Zone de texte</div>
<script>
      document.getElementById("text").innerHTML="Ceci est notre texte"
   </script>
</body>
</html>>


In [4]:

print(soup.title.get_text())
print(soup.find(id='text').get_text())
soup_file.close()

Page Dynamique
Zone de texte


Dans le code ci-dessus, nous ouvrons le fichier html et par la suite nous utilisons BS pour parcourir le code source de la page.

Mais comme vous pouvez le voir, nous n'avons récupéré que le contenu static de la page. Le texte généré par le JS n'as pu etre récupéré par BS. **C'est là que selenium rentre en jeu**

## SELENIUM

Selenium est un outil d’automatisation de test pour le web. Il permet de créer des **robots** qui naviguent dans des pages webs comme le ferait un vrai utilisateur. Bien que le premier rôle de Selenium soit le testing de pages webs (développement web), cet outil est beaucoup utilisé pour l’extraction de données.

## Pourquoi utiliser Selenium pour scraper ?

   La démarche habituelle pour scraper est : une requête suivie du parsing de la réponse. Dans la plupart des cas cela marche bien. Mais lorsque l’on a affaire à des sites avec plusieurs redirections ou avec des pop-ups avec lesquels il faut interagir avant d’avoir la page qu’on veut, cette démarche devient beaucoup moins amusante. Car il faut analyser le réseau et simuler les requêtes dans le bon ordre avec les bons arguments. Et parfois il est carrement impossible d’accéder à une page via une simple requête.

   L’avantage de Selenium c’est que l’on (notre script) peut naviguer sur les pages. Du coup, si on voit la donnée dans notre navigateur, on peut la scraper via Selenium. Avec selenium, on peut remplir des formulaire, cliquer sur des boutons, scroller, parser des pages etc.

In [3]:
import os

In [4]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

In [5]:
os.getcwd()

'd:\\Data Science\\semestre9\\Webscraping\\SCRAPING_DYNAMIQUE'

In [7]:

from selenium.webdriver.firefox.options import Options

options = Options()
# gecko_driver_path = "../web_drivers/geckodriver.exe"
# Specifiez le chemin de votre Driver
options.binary_location = "C:/Program Files/Mozilla Firefox/firefox.exe"

# Création d'une instance driver
driver = webdriver.Firefox()

# Demander au driver d'appler sur Facebook
# driver.get("https://www.facebook.com/")

In [12]:
"\\"

'\\'

'file:///c:/Users/Utilisateur/Documents/FORMATIONS/WEB_SCRAPING/PRATIQUES_WEB_SCRAPING/SCRAPING_DYNAMIQUE/test.html'

In [8]:
# On va sur notre page test.html
# url = r"file:///C:/Users/Utilisateur/Documents/FORMATIONS/WEB_SCRAPING/PRATIQUES_WEB_SCRAPING/SCRAPING_DYNAMIQUE/test.html"
dirPath = str(os.getcwd() + "\\test.html")
dirPath = dirPath.replace("\\", "/")
url = r"file:///"+dirPath
print(url)
driver.get(url)


file:///d:/Data Science/semestre9/Webscraping/SCRAPING_DYNAMIQUE/test.html


In [26]:
# import time

# time.sleep(10)
# driver.quit()


L'étape suivante consiste à créer un objet BeautifulSoup et à y charger la source de la page. Nous pouvons ensuite extraire les données de cette source. Dans le code ci-dessous, vous pouvez voir que nous faisons à peu près la même chose que dans l'exercice précédent. Mais cette fois, le résultat sera différent

In [9]:
soup_file = driver.page_source
soup_file

'<html><head>\n   <title>Page Dynamique</title>\n</head>\n<body>\n   <div id="text">Ceci est notre texte</div>\n   <script>\n      document.getElementById("text").innerHTML="Ceci est notre texte"\n   </script>\n\n</body></html>'

In [10]:
# Mettez la source de la page dans une variable et créez un objet BS à partir de celle-ci 
soup_file = driver.page_source 
soup = BeautifulSoup(soup_file)
# Charger et imprimer le titre et le texte de <div> 
print (soup.title.get_text ()) 
print(soup.find(id='text').get_text())

Page Dynamique
Ceci est notre texte


In [11]:
soup.prettify

<bound method Tag.prettify of <html><head>
<title>Page Dynamique</title>
</head>
<body>
<div id="text">Ceci est notre texte</div>
<script>
      document.getElementById("text").innerHTML="Ceci est notre texte"
   </script>
</body></html>>

Comme vous pouvez le voir, nous utilisons maintenant le code qui a été traité par le navigateur Web driver, le résultat est ce qui serait rendu dans une fenêtre de navigateur, et non la source d'origine comme lors de notre première tentative.

In [12]:
# Fermez le navigateur 
driver.quit ()

## Exemple

Nous voulons avoir les prix, titres d’offre et images pour toutes les offres concernant les IPhones sur le site de [https://www.ivoiremobiles.net/]("https://www.ivoiremobiles.net/"). Pour cela nous allons juste faire une recherche du mot clé “iphone” puis récupérer les informations. Simple non ?

In [14]:
#  Le webdriver est l’objet qui gère le navigateur utilisé par Selenium
from selenium import webdriver

# Pour utiliser toutes les touches du clavier
from selenium.webdriver.common.keys import Keys

# Pour mettre le script en “pause” pour les chargement de page.
import time

# Accéder aux options du driver Selenium
from selenium.webdriver.firefox.options import Options

# Pour exécuter le navigateur en arrière-plan
# options.add_argument("--headless")

options = Options()
# Specifiez le chemin de votre Driver
options.binary_location = "C:/Program Files/Mozilla Firefox/firefox.exe"

# Création d'une instance driver
driver = webdriver.Firefox()


In [17]:
#gecko_driver_path = "put_path_to_your_driver.exe"
# gecko_driver_path = "../web_drivers/geckodriver.exe"

# Création d'une instance driver
# driver = webdriver.Firefox(executable_path=gecko_driver_path)

# Chargement de la page dans le driver
url = "https://www.ivoiremobiles.net"
driver.get(url)

# En fonction de notre connexion et des performance de notre machine il faudra attendre
# que la page charge avant de passer à la suite
time.sleep(10)

On recupère, grâce au selecteur de Selenium, l’élément qui correspond au champ “recherche”. Pour trouver le bon élément il faut inspecter la page

In [23]:
from selenium.webdriver.common.by import By
# On recupere la bar de recherche, on la remplit avec "iphone" puis on appuie "Entrez"
# driver.refresh()
# search_bar = driver.find_element_by_name("search_query")
search_bar = driver.find_element(By.ID, "search_query_top")
search_bar.clear()
time.sleep(5)
search_bar.send_keys("iphone")




In [24]:
time.sleep(5)
search_bar.send_keys(Keys.ENTER)

Jetez maintenant un oeil sur le webdriver généré par selenium

In [25]:
# Chargement du code source de la page recherchée
page = driver.page_source
soup = BeautifulSoup(page)

In [26]:
soup.prettify

<bound method Tag.prettify of <html lang="fr"><head><meta charset="utf-8"/><title>Résultats de recherche pour "iphone"</title><meta content="PrestaShop" name="generator"/><meta content="index,follow" name="robots"/><meta content="width=device-width, minimum-scale=0.25, maximum-scale=1.6, initial-scale=1.0" name="viewport"/><meta content="yes" name="apple-mobile-web-app-capable"/><link href="/img/favicon.ico?1655247122" rel="icon" type="image/vnd.microsoft.icon"/><link href="/img/favicon.ico?1655247122" rel="shortcut icon" type="image/x-icon"/><link href="https://www.ivoiremobiles.net/themes/default-bootstrap/cache/v_72_4675ccdb673d8a3d1c88ec31261ede8c_all.css" media="all" rel="stylesheet" type="text/css"/><meta content="" property="retailer_item_id"/><meta content="" property="og:price"/><meta content="XOF" property="og:priceCurrency"/> <script src="https://pagead2.googlesyndication.com/pagead/managed/js/adsense/m202411140101/reactive_library_fy2021.js"></script><script async="" src="h

In [27]:
# La liste des articles (Iphone)
products = soup.find_all('div',class_='product-container')

In [28]:
len(products)

113

In [29]:
products[0]

<div class="product-container" itemscope="" itemtype="http://schema.org/Product"><div class="left-block"><div class="product-image-container"> <a class="product_img_link" href="https://www.ivoiremobiles.net/apple/1922-apple-iphone-15-pro-128-go.html" itemprop="url" title="Apple iPhone 15 Pro 128 Go"> <img alt="Apple iPhone 15 Pro 128 Go" class="replace-2x img-responsive" height="250" itemprop="image" src="https://www.ivoiremobiles.net/25766-home_default/apple-iphone-15-pro-128-go.jpg" title="Apple iPhone 15 Pro 128 Go" width="250"/> </a><div class="content_price" itemprop="offers" itemscope="" itemtype="http://schema.org/Offer"> <span class="price product-price" itemprop="price"> 619 900 FCFA </span><meta content="XOF" itemprop="priceCurrency"/></div></div></div><div class="right-block"><h5 itemprop="name"> <a class="product-name" href="https://www.ivoiremobiles.net/apple/1922-apple-iphone-15-pro-128-go.html" itemprop="url" title="Apple iPhone 15 Pro 128 Go"> Apple iPhone 15 Pro 128 Go

In [30]:
soup.find()

<html lang="fr"><head><meta charset="utf-8"/><title>Résultats de recherche pour "iphone"</title><meta content="PrestaShop" name="generator"/><meta content="index,follow" name="robots"/><meta content="width=device-width, minimum-scale=0.25, maximum-scale=1.6, initial-scale=1.0" name="viewport"/><meta content="yes" name="apple-mobile-web-app-capable"/><link href="/img/favicon.ico?1655247122" rel="icon" type="image/vnd.microsoft.icon"/><link href="/img/favicon.ico?1655247122" rel="shortcut icon" type="image/x-icon"/><link href="https://www.ivoiremobiles.net/themes/default-bootstrap/cache/v_72_4675ccdb673d8a3d1c88ec31261ede8c_all.css" media="all" rel="stylesheet" type="text/css"/><meta content="" property="retailer_item_id"/><meta content="" property="og:price"/><meta content="XOF" property="og:priceCurrency"/> <script src="https://pagead2.googlesyndication.com/pagead/managed/js/adsense/m202411140101/reactive_library_fy2021.js"></script><script async="" src="https://www.google-analytics.co

In [31]:
# Avoir la balise contenant la description de l'article
products[0].find("h5",itemprop='name')

<h5 itemprop="name"> <a class="product-name" href="https://www.ivoiremobiles.net/apple/1922-apple-iphone-15-pro-128-go.html" itemprop="url" title="Apple iPhone 15 Pro 128 Go"> Apple iPhone 15 Pro 128 Go </a></h5>

In [32]:
products[0].find("h5", itemprop='name').get_text()

'  Apple iPhone 15 Pro 128 Go '

In [37]:
# Le prix de l'article
prix=products[0].find("span", itemprop='price').get_text()
prix

' 619 900 FCFA '

In [38]:
# L'image de l'article
image=products[0].find("img", itemprop='image')
image

<img alt="Apple iPhone 15 Pro 128 Go" class="replace-2x img-responsive" height="250" itemprop="image" src="https://www.ivoiremobiles.net/25766-home_default/apple-iphone-15-pro-128-go.jpg" title="Apple iPhone 15 Pro 128 Go" width="250"/>

In [35]:
# Le lien de l'image
products[0].find("img", itemprop='image')['src']

'https://www.ivoiremobiles.net/25766-home_default/apple-iphone-15-pro-128-go.jpg'

In [39]:
# Le titre de l'image
article=products[0].find("img", itemprop='image')['title']
article

'Apple iPhone 15 Pro 128 Go'

In [40]:
print(f'Article |Prix| Image| Desc Image\n')
print(f'{article}|{prix}|{image}')

Article |Prix| Image| Desc Image

Apple iPhone 15 Pro 128 Go| 619 900 FCFA |<img alt="Apple iPhone 15 Pro 128 Go" class="replace-2x img-responsive" height="250" itemprop="image" src="https://www.ivoiremobiles.net/25766-home_default/apple-iphone-15-pro-128-go.jpg" title="Apple iPhone 15 Pro 128 Go" width="250"/>


Parcourons maintenant la liste des articles pour avoir les infos sur tous les articles

In [41]:
import pandas as pd
names = []
prices = []
links = []

for prd in products:
        names.append(prd.find("h5", itemprop='name').get_text())
        prices.append(prd.find("span", itemprop='price').get_text())
        links.append(prd.find("img", itemprop='image')["src"])
        
df_iphones = pd.DataFrame({"names":names, "prices":prices, "link_image":links})

In [42]:
df_iphones.head(10)

,names,prices,link_image
0,Apple iPhone 15 Pro 128 Go,619 900 FCFA,https://www.ivoiremobiles.net/25766-home_defau...
1,Apple iPhone 15 Pro Max 256 Go,759 900 FCFA,https://www.ivoiremobiles.net/25802-home_defau...
2,Apple iPhone 15 Pro Max 512 Go,859 900 FCFA,https://www.ivoiremobiles.net/25803-home_defau...
3,Apple iPhone 15 Pro Max 1 To,1 029 900 FCFA,https://www.ivoiremobiles.net/25821-home_defau...
4,Apple iPhone 15 Pro 256 Go,689 900 FCFA,https://www.ivoiremobiles.net/26125-home_defau...
5,Apple iPhone 15 Pro 512 Go,849 900 FCFA,https://www.ivoiremobiles.net/26144-home_defau...
6,Apple iPhone 15 Pro 1 To,929 900 FCFA,https://www.ivoiremobiles.net/26162-home_defau...
7,Apple iPhone 11 Pro 64 Go,329 900 FCFA,https://www.ivoiremobiles.net/23763-home_defau...
8,Apple iPhone 11 Pro Max 64 Go,589 900 FCFA,https://www.ivoiremobiles.net/15868-home_defau...
9,Apple iPhone 11 Pro 256 Go,359 900 FCFA,https://www.ivoiremobiles.net/23764-home_defau...


Nous avons maintenant un dataframe contenant tous les téléphones Iphone disponible sur le site

In [59]:
df_iphones.to_csv("./DATA/iphones.csv", sep = ";", index = False)